In [1]:
!pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/drive/MyDrive/knowledge injection for metaphor identification sdllod23/datasets/datasets/VUA-V/test.csv',
                   sep=',',  index_col=False)

In [5]:
#0. Imports
import json
import copy
import sys

from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON
from SPARQLWrapper.Wrapper import QueryResult

from rdflib import Namespace, Literal, Graph, URIRef
from rdflib.namespace import RDF

In [6]:
from nltk.wsd import lesk
import nltk
import re
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
def FS_query(target_word):
  '''use this code to access single or small amount of words (limit 10000)'''
  sparql = SPARQLWrapper("http://etna.istc.cnr.it/framester2/sparql")
  sparql.setReturnFormat('json')
  sparql.setQuery("""
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
      PREFIX framestercore: <https://w3id.org/framester/data/framestercore/>
      PREFIX fschema: <https://w3id.org/framester/schema/>
      PREFIX abox: <https://w3id.org/framester/framenet/abox/frame/>
      SELECT DISTINCT ?fscf WHERE {
      # ?fscf a rdfs:Class .
      ?fscf skos:closeMatch """+target_word+""" .
      # <https://w3id.org/framester/wn/wn30/instances/synset-borrow-verb-1> a  <https://w3id.org/framester/wn/wn30/schema/VerbSynset> .
      ?fscf <https://w3id.org/framester/framenet/tbox/frame_name> ?name .
      ?fscf  <https://w3id.org/framester/framenet/tbox/hasFrameElement> ?fe .
      } LIMIT 40
  """)
  return(QueryResult(sparql._query()))._convertJSON()


In [8]:
def disambiguate(sentence, target_word, pos):
  synset = lesk(sentence, target_word, pos)
  template = '<https://w3id.org/framester/wn/wn30/instances/synset-<label>-<pos>-<n>>'
  label = str(synset).split("'")[1].split('.')[0]
  pos = 'verb'
  n = str(synset).split("'")[1].split('.')[2][1]
  sentence = re.sub("<label>", label, template)
  sentence = re.sub("<pos>", pos, sentence)
  sentence = re.sub("<n>", n, sentence)
  return(sentence)

In [9]:
disambiguate('Design : Crossed lines over the toytown tram : City transport could soon be back on the right track , says Jonathan Glancey', 'cross', 'v')

'<https://w3id.org/framester/wn/wn30/instances/synset-intersect-verb-1>'

In [9]:
FS_query('<https://w3id.org/framester/wn/wn30/instances/synset-lend-verb-2>')

{'head': {'link': [], 'vars': ['fscf']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'fscf': {'type': 'uri',
     'value': 'https://w3id.org/framester/framenet/abox/frame/People'}}]}}

In [ ]:
#now link the data
df['injected_knowledge'] = 1
for i in range(len(df['target_word'])):
  try:
    d_list = []
    word = df['target_word'].iloc[i]
    sentence = df['sentence'].iloc[i]
    sense = disambiguate(sentence, word, 'v')
    FS_res = FS_query(sense)
    for j in FS_res['results']['bindings']:
      d_list.append(j['fscf']['value'].split('/')[-1])
    #ik = FS_res['results']['bindings'][0]['fscf']['value']
    df['injected_knowledge'].iloc[i] = d_list
  except:
    #print('no info for:'+ df['target_word'].iloc[i])

<ipython-input-10-74fa3d01f33d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['injected_knowledge'].iloc[i] = d_list


no info for:flat
no info for:widespread
no info for:algae
no info for:half
no info for:co
no info for:determ
no info for:re
no info for:bogg
no info for:polychlorinat
no info for:polychlorinat
no info for:misdescribed
no info for:gon
no info for:gon
no info for:er
no info for:gon
no info for:int
no info for:er
no info for:erm
no info for:gon
no info for:aye
no info for:gon
no info for:gobsmack
no info for:er
no info for:gon
no info for:gon
no info for:gon
no info for:forty
no info for:ha
no info for:mum
no info for:gon
no info for:cos
no info for:gon
no info for:shall
no info for:gon
no info for:gon
no info for:think
no info for:know
no info for:think
no info for:open
no info for:please
no info for:hold
no info for:come
no info for:get
no info for:get
no info for:turn
no info for:go
no info for:get
no info for:wait
no info for:wait
no info for:drop
no info for:watch
no info for:come
no info for:get
no info for:think
no info for:call
no info for:shut
no info for:try
no info for:get
no i

In [34]:
df

,sentence,label,target_position,target_word,pos_tag,gloss,eg_sent,In_Know,injected_knowledge
0,Design : Crossed lines over the toytown tram :...,1,2,cross,VERB,Go or extend across or to the other side of (a...,she has crossed the Atlantic twice,1,"[https:, , w3id.org, framester, framenet, abox..."
1,Design : Crossed lines over the toytown tram :...,0,20,say,VERB,"Utter words so as to convey information, an op...","‘Thank you,’ he said",1,"[https:, , w3id.org, framester, framenet, abox..."
2,"MODERN trams , as most continental Europeans k...",0,7,know,VERB,"Be aware of through observation, inquiry, or i...",most people know that CFCs can damage the ozon...,1,1
3,"MODERN trams , as most continental Europeans k...",0,10,shake,VERB,(of a structure or area of land) tremble or vi...,buildings shook in Sacramento and tremors were...,1,1
4,"MODERN trams , as most continental Europeans k...",0,12,rattle,VERB,Make or cause to make a rapid succession of sh...,the roof rattled with little gusts of wind,1,1
...,...,...,...,...,...,...,...,...,...
5868,Aristotle said something very interesting in t...,1,1,say,VERB,"Utter words so as to convey information, an op...","‘Thank you,’ he said",1,1
5869,Aristotle said something very interesting in t...,0,13,quote,VERB,Repeat or copy out (words from a text or speec...,I realized she was quoting passages from Shake...,1,1
5870,Aristotle said something very interesting in t...,1,1,say,VERB,"Utter words so as to convey information, an op...","‘Thank you,’ he said",1,1
5871,Aristotle said something very interesting in t...,0,27,lack,VERB,Be without or deficient in.,the novel lacks imagination,1,1
